In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
aapl = np.flip(np.array(yf.Ticker("AAPL").history(period="3mo").Close))
amzn = np.flip(np.array(yf.Ticker("AMZN").history(period="3mo").Close))
nke = np.flip(np.array(yf.Ticker("NKE").history(period="3mo").Close))
mcd = np.flip(np.array(yf.Ticker("MCD").history(period="3mo").Close))
koss = np.flip(np.array(yf.Ticker("KOSS").history(period="3mo").Close))
ko = np.flip(np.array(yf.Ticker("KO").history(period="3mo").Close))
pg = np.flip(np.array(yf.Ticker("PG").history(period="3mo").Close))
eman = np.flip(np.array(yf.Ticker("EMAN").history(period="3mo").Close))
ba = np.flip(np.array(yf.Ticker("BA").history(period="3mo").Close))
msft = np.flip(np.array(yf.Ticker("MSFT").history(period="3mo").Close))

data = np.array([aapl, amzn, nke, mcd, koss, ko, pg, eman, ba, msft])

### Alphas from Zura Kakushadze, 101 alphas, 2015

##### Alpha#1

In [3]:
def f_returns(data, comp_id):
    return [data[comp_id][t] / data[comp_id][t + 1] - 1 for t in range(data.shape[1] - 1)]

In [4]:
def f_stddev(returns):
    return np.std(returns)

In [5]:
def kernel(data, comp_id):
    if data[comp_id][0] < data[comp_id][1]:
        return np.power(f_stddev(f_returns(data, comp_id)), 2)
    else:
        return np.power(data[comp_id][0], 2)


In [6]:
def ts_argmax(data, comp_id):
    ts = [kernel(np.lib.stride_tricks.sliding_window_view(data[comp_id], 20)[:5], i) for i in range(5)]
    return np.argmax(ts)

In [7]:
def alpha1(data):
    rank = stats.rankdata([ts_argmax(data, i) for i in range(len(data))], method='ordinal') / len(data)
    alpha = (rank - np.mean(rank)) / sum(abs(rank - np.mean(rank)))
    return alpha

In [8]:
alphas_dataframe = pd.DataFrame()
alphas_dataframe = alphas_dataframe.reindex(columns=alphas_dataframe.columns.tolist() + ['aapl', 'amzn', 'nke', 'mcd', 'koss', 'ko', 'pg', 'eman', 'ba', 'msft'])
alphas_dataframe.loc[len(alphas_dataframe.index)] = alpha1(data)
alphas_dataframe

,aapl,amzn,nke,mcd,koss,ko,pg,eman,ba,msft
0,-0.18,0.14,-0.02,0.18,-0.14,0.02,0.06,-0.1,0.1,-0.06
